### Recall

In [ ]:
def load_embeddings(file_path):
    """
    从给定路径的文件中加载嵌入向量，并将它们存储在字典中。

    :param file_path: 带有嵌入向量的文件的路径
    :return: 字典，其中键是item_id，值是对应的向量列表
    """
    # 创建一个空字典来存储item_id和相应的向量
    embeddings = {}
    # 打开文件进行读取
    with open(file_path, 'r') as file:
        # 按行读取数据
        for line in file:
            # 移除行尾的换行符和额外的空格，并按":"进行分割
            item_id, vector_str = line.strip().split(':')
            # 将字符串形式的向量转换为浮点数列表
            vector = [float(x) for x in vector_str.split()]
            # 将item_id和向量添加到字典中
            embeddings[item_id] = vector
            
    return embeddings

In [ ]:
file_path = 'emb/item2vecEmb.csv'
item_embeddings = load_embeddings(file_path)

In [ ]:
import numpy as np

def recallCandidates(user_vector, embedding_matrix, item_ids, top_n):
    """
    根据用户向量和项目嵌入矩阵来回调最相似的top n个项目。

    :param user_vector: 一维数组，表示用户的向量
    :param embedding_matrix: 二维数组，其中每行代表一个项目的嵌入向量
    :param item_ids: 列表数据，包含与embedding_matrix中行顺序相对应的项目ID
    :param top_n: 整型数据，指定需要返回的最相似项目的数量

    :return: 包含Top n候选项目的item_ids和它们的余弦相似度得分的列表
    """
    
    # 确认用户向量和项目向量的维度匹配
    assert user_vector.size == embedding_matrix.shape[1]
    
    # 计算用户向量与所有项目向量的点积
    dot_product = np.dot(embedding_matrix, user_vector)
    
    # 计算用户向量和所有项目向量的模长
    user_vector_length = np.linalg.norm(user_vector)
    item_vector_lengths = np.linalg.norm(embedding_matrix, axis=1)
    
    # 计算余弦相似度
    cosine_similarity = dot_product / (user_vector_length * item_vector_lengths)
    
    # 获取相似度最高的前top_n个项目的索引
    top_n_indices = np.argsort(-cosine_similarity)[:top_n]
    
    # 获取对应的余弦相似度得分
    top_n_cosine_similarity_scores = cosine_similarity[top_n_indices]
    
    # 根据索引获取对应的item_id
    top_n_item_ids = [item_ids[i] for i in top_n_indices]
    
    # 结合item_id和它们的余弦相似度得分
    top_n_candidates = [(item_ids[index], score) for index, score in zip(top_n_indices, top_n_cosine_similarity_scores)]
    
    return top_n_candidates

# 使用示例
# item_ids = ... # 从中获取你的item_ids的代码
# user_vector = ... # 用户的向量
# embedding_matrix = ... # 嵌入矩阵
# top_n = 10 # 想要回调的候选项目数量

# 假设item_ids和embedding_matrix已通过某种方式得到
# 可以调用recallCandidates函数获得结果
top_n =10
user_vector = np.array([0.1, 0.3, 0.5, 0.1, 0.3, 0.5, 0.1, 0.3, 0.5, 0.1])  # 示例用户向量
embedding_matrix = np.array(list(item_embeddings.values()))
item_ids = list(item_embeddings.keys())

candidates = recallCandidates(user_vector, embedding_matrix, item_ids, top_n)
for item_id, cos_sim in candidates:
    print(f"Item ID: {item_id}, Cosine Similarity: {cos_sim}")

### NeuralCF

In [ ]:
import tensorflow as tf

saved_model_path = "model/NeuralCF"

# 加载模型
model = tf.keras.models.load_model(saved_model_path)

In [ ]:
# 构建输入字典
predict_input = {
    'userId': np.array([1]), 
    'movieId': np.array([1])
}

# 使用模型进行预测
predicted_scores = model.predict(predict_input)
print(predicted_scores)

In [ ]:
user_emb_path = 'emb/item2vecEmb.csv'
user_embeddings = load_embeddings(user_emb_path)

In [ ]:
def makePredictInput(user_id, candidates):
    """
    创建模型预测所需的(userId, movieId)对。

    :param user_id: int, 需要生成配对的用户ID
    :param candidates: list of tuples, 每个元组包含 (movieId, score)
    :return: dict, 包含用于模型预测的输入数据，'userId'和'movieId'键的批次数组
    """
    # 提取所有的电影ID
    candidate_ids = [candidate[0] for candidate in candidates]
    
    # 用户ID数组，重复以匹配候选项目的数量
    user_ids = [user_id] * len(candidate_ids)

    # 构建预测输入字典
    predict_input = {
        'userId': np.array(user_ids, dtype=np.int32),     # 用户ID数组，int32类型
        'movieId': np.array(candidate_ids, dtype=np.int32)  # 电影ID数组，int32类型
    }
    
    return predict_input

for user_id in user_embeddings.keys():
    user_vector = np.array(user_embeddings[user_id])
    candidates = recallCandidates(user_vector, embedding_matrix, item_ids, top_n)
    predict_input = makePredictInput(user_id, candidates)
    predicted_scores = model.predict(predict_input, verbose=0)
    flat_scores = [score[0] for score in predicted_scores]